# Import Required Libraries
Import the necessary libraries, such as requests, for making API calls.

In [ ]:
# Import the necessary library for making API calls
#%pip install requests
#%pip install pandas
#%pip install datetime
#%pip install os
#%pip install cudf
#%pip install cupy
#%pip install openpyxl
#%pip install textblob
#%pip install transformers
#%pip install torch
#%pip install import-ipynb
import import_ipynb
from Stock_Reserach import *
import requests
import pandas as pd
import datetime as dt
import os
from textblob import TextBlob
from transformers import pipeline

c:\Users\ravit\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Running correlation between all the stocks in the exchange
Creates a list of all the correlations betweens stocks and sorts them

In [ ]:
def calculate_and_sort_correlations(tickers_df, exchange, instrument_type="Common Stock", api_token=API_Eodhd, start_date="2025-01-01", end_date="2025-03-30", period="d"):
    """
    Calculate the correlation between all stocks in an exchange and sort them from least to highest.

    Parameters:
        tickers_df (pd.DataFrame): DataFrame containing stock tickers.
        exchange (str): Exchange code to append to tickers (e.g., "US").
        instrument_type (str): Type of the instrument to filter (e.g., "Common Stock").
        api_token (str): API token for authentication.
        start_date (str): Start date for historical data.
        end_date (str): End date for historical data.
        period (str): Period for historical data (e.g., "d" for daily).

    Returns:
        pd.DataFrame: DataFrame containing stock pairs and their correlation, sorted by correlation value.
    """
    # Filter tickers to include only the specified instrument type
    filtered_tickers_df = tickers_df[tickers_df["Type"] == instrument_type]

    stock_data = {}
    for ticker in filtered_tickers_df['Code']:
        try:
            # Combine ticker with exchange code
            full_ticker = f"{ticker}.{exchange}"
            stock_df = Fetch_historical_price(full_ticker, start_date, end_date, period, api_token, columns=["adjusted_close"])
            stock_data[ticker] = stock_df.set_index("date")["adjusted_close"]
        except Exception as e:
            print(f"Error fetching data for {ticker}: {e}")

    # Combine all stock data into a single DataFrame
    combined_df = pd.DataFrame(stock_data)

    # Calculate correlation matrix
    correlation_matrix = combined_df.corr()

    # Create a list of all pairs and their correlations
    correlations = []
    for i in range(len(correlation_matrix.columns)):
        for j in range(i + 1, len(correlation_matrix.columns)):
            stock1 = correlation_matrix.columns[i]
            stock2 = correlation_matrix.columns[j]
            correlation = correlation_matrix.iloc[i, j]
            correlations.append({"Instrument 1": stock1, "Instrument 2": stock2, "Correlation": correlation})

    # Convert the list of correlations to a DataFrame
    correlations_df = pd.DataFrame(correlations)

    # Sort the DataFrame by correlation value
    correlations_df = correlations_df.sort_values(by="Correlation", ascending=True).reset_index(drop=True)

    return correlations_df

# Example usage
tickers = fetch_exchange_symbols("AS")
tickers_subset = tickers
sorted_correlations_df = calculate_and_sort_correlations(tickers_subset, "AS", instrument_type="ETF")
print(sorted_correlations_df)

# Exporting output to excel
Creates an excel export in the python folder

In [ ]:
# Define the output directory and file path
output_dir = r"E:\Business NL\Python\Excel output"
os.makedirs(output_dir, exist_ok=True)  # Ensure the directory exists
output_file = os.path.join(output_dir, "sorted_correlations.xlsx")

# Export the DataFrame to an Excel file
sorted_correlations_df.to_excel(output_file, index=False)

print(f"Sorted correlations exported to {output_file}")

# Analyzing sentement with finbert model
Creates a function to use finbert model to analyze sentement

In [ ]:

def analyze_sentiment_with_finbert(news_df):
    """
    Analyze the sentiment of news articles using FinBERT.

    Parameters:
        news_df (pd.DataFrame): DataFrame containing news articles.

    Returns:
        pd.DataFrame: DataFrame with an additional 'Sentiment' column.
    """
    if 'content' not in news_df.columns:
        raise ValueError("The DataFrame must contain a 'content' column for sentiment analysis.")
    
    # Load the FinBERT sentiment analysis pipeline with explicit truncation
    finbert = pipeline(
        "sentiment-analysis",
        model="yiyanghkust/finbert-tone",
        tokenizer="yiyanghkust/finbert-tone",
        device=0,  # Use CPU (-1) or GPU (0 or higher)
        truncation=True,
        max_length=512  # Explicitly set the maximum length
    )
    
    # Apply sentiment analysis to the 'content' column
    def analyze_text(text):
        try:
            return finbert(text[:512])[0]['label']  # Truncate text to 512 characters
        except Exception as e:
            return f"Error: {e}"
    
    news_df['Sentiment'] = news_df['content'].apply(analyze_text)
    
    return news_df

# Example usage
api_token = API_Eodhd  # Replace with your actual API token
stock = "AAPL.US"
begin_date = "2023-01-01"
end_date = "2025-03-01"
tag = "balance sheet"

# Fetch news data
news_df = fetch_news_data(stock, tag, begin_date, end_date, api_token=api_token)

# Perform sentiment analysis with FinBERT
news_with_sentiment = analyze_sentiment_with_finbert(news_df)

print(news_with_sentiment)

Device set to use cpu


                        date  \
0  2023-03-24T13:00:11+00:00   
1  2023-02-17T11:01:03+00:00   
2  2023-02-14T14:24:02+00:00   
3  2023-01-26T13:06:00+00:00   
4  2023-01-18T11:00:49+00:00   
5  2023-01-14T12:04:45+00:00   
6  2023-01-01T23:08:43+00:00   

                                               title  \
0  Apple (NASDAQ:AAPL) Seems To Use Debt Quite Se...   
1        Is Apple (NASDAQ:AAPL) Using Too Much Debt?   
2  This Semiconductor Stock Has Too Much Dependen...   
3  If Cash Is King, These Nasdaq Stocks Reign Sup...   
4  Apple (NASDAQ:AAPL) Could Easily Take On More ...   
5            7 Stocks to Buy and Hold For a Lifetime   
6  Lessons Learned From Visa and Mastercard in a ...   

                                             content  \
0  Howard Marks put it nicely when he said that, ...   
1  David Iben put it well when he said, 'Volatili...   
2  In today's video, Jose Najarro, Nick Rossolill...   
3  Because of that, companies that generate lots ...   
4  Howard Mark